### Read the station data from GHCN archive ###

Data obtained from http://www.ncdc.noaa.gov/cdo-web/

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
%matplotlib inline

In [2]:
def get_date(date_number):
    """
    Turn the int64 value from the DATE of GHCN into a pd.datetime
    """
    dstring = str(date_number)
    return pd.datetime(int(dstring[0:4]),int(dstring[4:6]),int(dstring[6:8]))

def get_df(fnm, var):
    """
    Create a dataframe for a single station, with a time index, for a single
    variable of data given as a key word (e.g. PRECIP, TMAX, TMIN).
    Requires file path and name (fnm).
    """
    #var = "PRCP"
    df = pd.read_csv(flist[0])
    dt_indx = [get_date(date) for date in df.DATE]   
    return pd.DataFrame(data=df[var].values,index=dt_indx,columns=[df.STATION[0][6:]])

In [3]:
fpth = "Data/*.csv"
flist = glob.glob(fpth)

df_dic = {}
for f in flist:
    df_dic[f[5:]] = get_df(fnm = f, var = 'PRCP')

In [30]:
df_dic  # <- a Dictionary of dataframes, all with time as the index, ready to be combined!
# Now you need to use the pd.concat command on the DICTIONARY of DATAFRAMES... 

In [23]:
df_dic.keys()

['598371.csv',
 '598366.csv',
 '598378.csv',
 '598377.csv',
 '598368.csv',
 '598374.csv',
 '598384.csv',
 '598385.csv',
 '598381.csv',
 '598391.csv',
 '598376.csv',
 '598362.csv',
 '598370.csv',
 '598375.csv',
 '598134.csv',
 '598382.csv',
 '598388.csv',
 '598367.csv',
 '598380.csv',
 '598386.csv',
 '598361.csv',
 '598365.csv']

In [28]:
#df_dic['598134.csv']

In [29]:
#df_dic['598361.csv']

In [25]:
pd.concat([df_dic['598371.csv'], df_dic['598366.csv'], df_dic['598378.csv'] ], axis=1)

,KE000063661,KE000063661,KE000063661
2000-01-01,0,0,0
2000-01-02,-9999,-9999,-9999
2000-01-03,-9999,-9999,-9999
2000-01-04,-9999,-9999,-9999
2000-01-05,0,0,0
2000-01-06,-9999,-9999,-9999
2000-01-08,-9999,-9999,-9999
2000-01-09,-9999,-9999,-9999
2000-01-10,-9999,-9999,-9999
2000-01-11,-9999,-9999,-9999


In [ ]:
#plt.plot(df_dic)

In [ ]:
#prcp.keys()

In [ ]:
#df = pd.read_csv("Data/598361.csv")

In [ ]:
#df.keys()

In [ ]:
#df.head()

In [ ]:
# Exaple of how to mask data and quick look
#mask = df.PRCP != -9999
#plt.plot(df.PRCP[mask])
#plt.show()

In [ ]:
# How to extract the head of the data
#df.DATE.head()

In [ ]:
#print(df.DATE[0])
#print(type(df.DATE[0]))
#df.DATE[0]

In [ ]:
#mask = mydata.KE000063820 != -9999
#plt.plot(mydata.index[mask],mydata.KE000063820[mask])
#plt.ylabel(r"$\frac{1}{10}$mm day$^{-1}$")

In [ ]:
#pd.concat([mydata.KE000063820,mydata.KE000063820,mydata.KE000063820, mydata.KE000063820],axis=1)

#mydata